In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import datetime
from pprint import pprint
import csv

In [2]:
data=pd.read_csv("cowin_vaccine_data_districtwise.csv")
data=data.iloc[:,:2116]

In [3]:
data=data.drop(data.index[0])
# data

In [4]:
data.drop(data.columns[[0,1, 4, 5]], axis = 1, inplace = True)
col = data.columns.drop(['District_Key','State'])
data[col] = data[col].apply(pd.to_numeric, errors='coerce')
data=data.replace(np.nan, 0)
# data

In [5]:
start=datetime.datetime(2021, 1, 16)
end=datetime.datetime(2021, 8, 14)
index=pd.date_range(start, end)
columns=['District_Key','State']
for item in index:
    column1=str(item.strftime('%d/%m/%Y'))+".3"
    column2=str(item.strftime('%d/%m/%Y'))+".4"
    columns.extend([column1,column2])
data = data[columns]  


In [6]:
district_data = data.drop(columns='State')
# district_data

In [7]:
data_dict=district_data.set_index('District_Key').T.to_dict('list')
dose1={}
dose2={}
for i in data_dict:
    dose1[i]=data_dict[i][::2]
    dose2[i]=data_dict[i][1::2]

In [8]:
state_week_doses=[]
district_week_doses=[]

for i in range(0,30):
    j={}
    for value in dose1:
        key=value.partition("_")[0]
        if key not in j:
            j[key]=[0,0]
        
        a=(dose1[value][i*7+7])-(dose1[value][i*7])
        b=(dose2[value][i*7+7])-(dose2[value][i*7])
        j[key][0]+=a
        j[key][1]+=b
        district_week_doses.append((value,i+1,a,b))
    for item in j:
        state_week_doses.append((item,i+1,j[item][0],j[item][1]))

# pprint(district_week_doses)

In [9]:
# pprint(state_week_doses)

In [10]:
district_week_doses.sort(key=lambda y: y[0])

with open('district-vaccinated-count-week.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['districtid','weekid','dose1','dose2'])
    for row in district_week_doses:
          csv_out.writerow((row[0],row[1],int(row[2]),(int(row[3]))))

In [11]:
state_week_doses.sort(key=lambda y: y[0])

with open('state-vaccinated-count-week.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['stateid','weekid','dose1','dose2'])
    for row in state_week_doses:
          csv_out.writerow((row[0],row[1],int(row[2]),(int(row[3]))))

## Overall

In [12]:
district_doses = district_data[['District_Key','14/08/2021.3','14/08/2021.4']].copy()
district_doses.columns=['districtid','dose1','dose2']
district_doses.to_csv('district-vaccinated-count-overall.csv')

In [13]:
state_overall_doses = data[['State','14/08/2021.3','14/08/2021.4']].copy()
aggregation = {'14/08/2021.3': 'sum', '14/08/2021.4': 'sum'}

state_overall_doses = state_overall_doses.groupby(['State'],as_index=False).aggregate(aggregation).reindex(columns=state_overall_doses.columns)



In [14]:
state_overall_doses.columns=['state','dose1','dose2']
state_overall_doses.to_csv('state-vaccinated-count-overall.csv')

## Monthly 

In [15]:
state_month_doses=[]
district_month_doses=[]

for i in range(0,7):
    j={}
    for value in dose1:
        key=value.partition("_")[0]
        if key not in j:
            j[key]=[0,0]
        
        a=(dose1[value][i*30+30])-(dose1[value][i*30])
        b=(dose2[value][i*30+30])-(dose2[value][i*30])
        j[key][0]+=a
        j[key][1]+=b
        district_month_doses.append((value,i+1,a,b))
    for item in j:
        state_month_doses.append((item,i+1,j[item][0],j[item][1]))

# pprint(district_month_doses)

In [16]:
district_month_doses.sort(key=lambda y: y[0])

with open('district-vaccinated-count-month.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['districtid','monthid','dose1','dose2'])
    for row in district_month_doses:
          csv_out.writerow((row[0],row[1],int(row[2]),(int(row[3]))))

In [17]:
state_month_doses.sort(key=lambda y: y[0])

with open('state-vaccinated-count-month.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['stateid','monthid','dose1','dose2'])
    for row in state_month_doses:
          csv_out.writerow((row[0],row[1],int(row[2]),(int(row[3]))))